In [97]:
# Importing Pywikibot may produce `Configuration variable ... is defined but unknown` warnings,
# but this is a false positive. It still authenticates properly.
import pywikibot as pwb
import datetime as dt
import re
import json

In [68]:
SURVEY_BASE_URL = "https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?"
PREVIEW_BASE_URL = "https://wikimedia.qualtrics.com/jfe/preview/SV_5ABs6WwrDHzAeLr?Q_CHL=preview&"

LIST_BASE = "Community Engagement Insights/MassMessages/Lists/2018/"
TEST_LIST_BASE = "User:Neil P. Quinn/2018 CE Insights/"

In [6]:
frame = pd.read_csv("sampling-frame.tsv", sep = "\t")
frame.head()

,user,home_proj,global_edits,proj_group,proj_domain
0,! Bikkit !,dewiki,219.0,dewiki,de.wikipedia.org
1,!NewLondon31,jawiki,46.0,jawiki,ja.wikipedia.org
2,!Silent,ptwiki,20688.8,ptwiki,pt.wikipedia.org
3,"""Colorado Campeão""!",ptwiki,417.0,ptwiki,pt.wikipedia.org
4,"""quasi"" tuttologo",itwiki,117.0,itwiki,it.wikipedia.org


# Set sample sizes

In [7]:
targets = pd.read_table("project-group-sample-targets.tsv").set_index("proj_group")
targets.head()

,sample_target
proj_group,
enwiki,900
arwiki,675
zhwiki,450
frwiki,450
eswiki,1125


In [8]:
# Round the highest edit count up to the nearest hundred thousand,
# in order to make a pretty edge for the top bin
top_edge = int(np.ceil(frame["global_edits"].max() / 100000) * 100000)
edit_bins = [10, 30, 150, 600, 1200, 3500, top_edge]
frame["binned_edits"] = pd.cut(frame["global_edits"], edit_bins, right=False)

In [9]:
strata_sizes = frame.groupby(["proj_group", "binned_edits"]).size().unstack()
group_sizes = pd.DataFrame(frame.groupby(["proj_group"]).count()["user"])
strata_group_pcts = np.divide(strata_sizes, group_sizes)
strata_targets = np.ceil(
    # Make sure we don't target fewer than 20 users per bin for privacy reasons
    np.maximum(
        np.multiply(strata_group_pcts, targets),
        20
    )
).apply(np.int64)
strata_samples = np.minimum(strata_targets, strata_sizes)

In [14]:
strata_samples

binned_edits,"[10, 30)","[30, 150)","[150, 600)","[600, 1200)","[1200, 3500)","[3500, 1100000)"
proj_group,,,,,,
arwiki,26,94,96,56,68,72
asia_wps,35,165,182,90,108,98
cee_wps,22,106,121,53,76,75
commons,20,53,84,51,88,166
dewiki,42,289,322,142,183,150
enwiki,28,143,133,49,54,46
eswiki,31,127,131,55,60,49
frwiki,65,298,303,137,166,159
itwiki,23,140,125,55,56,54


In [15]:
strata = pd.DataFrame(
    strata_samples.stack()
).reset_index()

strata.columns = ["proj_group", "edit_bin", "sample_size"]

strata.head()

,proj_group,edit_bin,sample_size
0,arwiki,"[10, 30)",26
1,arwiki,"[30, 150)",94
2,arwiki,"[150, 600)",96
3,arwiki,"[600, 1200)",56
4,arwiki,"[1200, 3500)",68


In [16]:
# Ignore any groups where we couldn't sample the minimum 20 users, because of re-identification risk
# In the 2018 edition, this is only 1 group
idx = strata.index[strata["sample_size"] < 20]

strata.loc[idx, "sample_size"] = 0

# Pull samples

In [84]:
def sample_stratum(stratum_query, sample_size):
    """
    returns the sampled users as a list of (user, proj_domain) tuples
    """ 
    stratum = frame.query(stratum_query)
    stratum_size = len(stratum)
    
    sample = stratum.sample(n = sample_size)[["user", "proj_domain"]]

    sample_list = [
        i for i in sample.itertuples(index = False, name = "user")
    ]

    return sample_list

In [18]:
def sample_row(row):
    query = "proj_group == '{pg}' & binned_edits == '{eb}'".format(
        pg = row.proj_group,
        eb = row.edit_bin
    )
    
    return sample_stratum(query, row.sample_size)

In [19]:
strata["sampled_users"] = strata.apply(sample_row, axis = 1)

In [20]:
strata.head()

,proj_group,edit_bin,sample_size,sampled_users
0,arwiki,"[10, 30)",26,"[(Michael M. Markos, ar.wikipedia.org), (Adam ..."
1,arwiki,"[30, 150)",94,"[(المسرحي, ar.wikipedia.org), (Ahmedsahhar, ar..."
2,arwiki,"[150, 600)",96,"[(Budur Subaie, ar.wikipedia.org), (Rofaida al..."
3,arwiki,"[600, 1200)",56,"[(Ahmed El-Farash, ar.wikipedia.org), (Faris k..."
4,arwiki,"[1200, 3500)",68,"[(Ibrahim Old, ar.wikipedia.org), (Moataz1997,..."


In [21]:
# Do all our strata have the correct number of sampled users?
(strata["sample_size"] == strata["sampled_users"].map(len)).value_counts()

True    108
dtype: int64

## Load corresponding test file

In [86]:
test_strata = pd.read_csv("test-strata-base.tsv", sep = "\t")
test_strata["sampled_users"] = test_strata["sampled_users"].map(json.loads)
test_strata.head()

,proj_group,edit_bin,sampled_users
0,arwiki,"[10, 30)","[[Neil P. Quinn-WMF, ar.wikipedia.org], [EGalv..."
1,asia_wps,"[30, 150)","[[Neil P. Quinn-WMF, ko.wikipedia.org], [EGalv..."
2,cee_wps,"[150, 600)","[[Neil P. Quinn-WMF, uk.wikipedia.org], [EGalv..."
3,commons,"[600, 1200)","[[Neil P. Quinn-WMF, commons.wikimedia.org], [..."
4,dewiki,"[1200, 3500)","[[Neil P. Quinn-WMF, de.wikipedia.org], [EGalv..."


# Link building

In [64]:
edit_codes = {
    "[10, 30)": 1,
    "[30, 150)": 2,
    "[150, 600)": 3,
    "[600, 1200)": 4,
    "[1200, 3500)": 5,
    "[3500, 1100000)": 6
}

proj_codes = {
    "enwiki": "en",
    "arwiki": "ar",
    "zhwiki": "zh",
    "frwiki": "fr",
    "eswiki": "es",
    "ruwiki": "ru",
    "dewiki": "de",
    "ptwiki": "pt",
    "nlwiki": "nl",
    "itwiki": "it",
    "jawiki": "ja",
    "meaf_wps": "me",
    "cee_wps": "ce",
    "asia_wps": "as",
    "weur_wps": "we",
    "commons": "co",
    "wikidata": "dt",
    "other": "ot"
}

In [65]:
def audience(edit_bin):
    if edit_bin <= 3:
        return "AE"
    else:
        return "VAE"

def build_url(row, base_url):
    extra_params = "aud={aud}&prj={prj}&edc={edc}&prjedc={prj}{edc}"
    prj = proj_codes[row.proj_group]
    edc = edit_codes[row.edit_bin]
    
    return base_url + extra_params.format(
        aud = audience(edc),
        prj = prj,
        edc = edc,
    )

def survey_url(row):
    return build_url(row, SURVEY_BASE_URL)

def preview_url(row):
    return build_url(row, PREVIEW_BASE_URL)

In [53]:
strata["survey_url"] = strata.apply(survey_url, axis = 1)
strata["preview_url"] = strata.apply(preview_url, axis = 1)

In [58]:
sample = strata.sample(n = 5)
for x in sample["survey_url"]:
    print(x)

for x in sample["preview_url"]:
    print(x)

https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=ar&edc=2&prjedc=ar2
https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=it&edc=2&prjedc=it2
https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=VAE&prj=es&edc=4&prjedc=es4
https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=ja&edc=3&prjedc=ja3
https://wikimedia.qualtrics.com/jfe/form/SV_5ABs6WwrDHzAeLr?aud=AE&prj=dt&edc=3&prjedc=dt3
https://wikimedia.qualtrics.com/jfe/preview/SV_5ABs6WwrDHzAeLr?Q_CHL=preview&aud=AE&prj=ar&edc=2&prjedc=ar2
https://wikimedia.qualtrics.com/jfe/preview/SV_5ABs6WwrDHzAeLr?Q_CHL=preview&aud=AE&prj=it&edc=2&prjedc=it2
https://wikimedia.qualtrics.com/jfe/preview/SV_5ABs6WwrDHzAeLr?Q_CHL=preview&aud=VAE&prj=es&edc=4&prjedc=es4
https://wikimedia.qualtrics.com/jfe/preview/SV_5ABs6WwrDHzAeLr?Q_CHL=preview&aud=AE&prj=ja&edc=3&prjedc=ja3
https://wikimedia.qualtrics.com/jfe/preview/SV_5ABs6WwrDHzAeLr?Q_CHL=preview&aud=AE&prj=dt&edc=3&prjedc=dt3


## Create test links

In [87]:
test_strata["survey_url"] = test_strata.apply(survey_url, axis = 1)
test_strata["preview_url"] = test_strata.apply(preview_url, axis = 1)

# Upload MassMessage lists

## Set up and test Pywikibot access 

In [22]:
# Pywikibot automatically authenticates our user-config.py file
# See https://www.mediawiki.org/wiki/Manual:Pywikibot/user-config.py
pwb_site = pwb.Site("meta", "meta")

def save_page(title, text, comment):
    page = pwb.Page(pwb_site, title)
    page.text = text
    page.save(comment, minor = False)

In [23]:
my_username = pwb_site.user()
test_title = "User:{user}/Pywikibot test".format(user = my_username)
test_text = "Pywikibot saved this page at " + str(dt.datetime.now()) + "."
test_comment = "Test Pywikibot access"

# If the page already exists, this will overwrite it 
save_page(test_title, test_text, test_comment)

Page [[User:WMF Surveys/Pywikibot test]] saved


## Set up and test target list generation

In [24]:
def list_text(users):
    text = ""
    
    for user in users:
        text += "* {{{{target | user = {} | site = {}}}}}\n".format(user[0], user[1])
        
    return text

In [94]:
def page_title(row, title_base):
    
    title = title_base + "{pg}{eb}"

    return title.format(
        pg = proj_codes[row.proj_group], 
        eb = edit_codes[row.edit_bin]
    )

def real_page_title(row):
    return page_title(row, LIST_BASE)

def test_page_title(row):
    return page_title(row, TEST_LIST_BASE)

In [45]:
strata["page_title"] = strata.apply(real_page_title, axis = 1)
strata.head()

,proj_group,edit_bin,sample_size,sampled_users,page_title
0,arwiki,"[10, 30)",26,"[(Michael M. Markos, ar.wikipedia.org), (Adam ...",Community Engagement Insights/MassMessages/Lis...
1,arwiki,"[30, 150)",94,"[(المسرحي, ar.wikipedia.org), (Ahmedsahhar, ar...",Community Engagement Insights/MassMessages/Lis...
2,arwiki,"[150, 600)",96,"[(Budur Subaie, ar.wikipedia.org), (Rofaida al...",Community Engagement Insights/MassMessages/Lis...
3,arwiki,"[600, 1200)",56,"[(Ahmed El-Farash, ar.wikipedia.org), (Faris k...",Community Engagement Insights/MassMessages/Lis...
4,arwiki,"[1200, 3500)",68,"[(Ibrahim Old, ar.wikipedia.org), (Moataz1997,...",Community Engagement Insights/MassMessages/Lis...


In [27]:
for row in strata.sample(n = 4).itertuples(index = False):
    print(row.page_title)
    print(list_text(row.sampled_users))

Community Engagement Insights/MassMessages/Lists/2018/jawiki - 600, 1200
* {{target | user = CAPRI-1 | site = ja.wikipedia.org}}
* {{target | user = 病は気から | site = ja.wikipedia.org}}
* {{target | user = 桐ヶ谷柩 | site = ja.wikipedia.org}}
* {{target | user = Strak Jegan | site = ja.wikipedia.org}}
* {{target | user = 月下薄氷 | site = ja.wikipedia.org}}
* {{target | user = パックリガム | site = ja.wikipedia.org}}
* {{target | user = Bird song | site = ja.wikipedia.org}}
* {{target | user = BPM123 | site = ja.wikipedia.org}}
* {{target | user = Triglav | site = ja.wikipedia.org}}
* {{target | user = 克己万否満 | site = ja.wikipedia.org}}
* {{target | user = フッキーフッキー | site = ja.wikipedia.org}}
* {{target | user = 空シュル | site = ja.wikipedia.org}}
* {{target | user = ニュートリノ | site = ja.wikipedia.org}}
* {{target | user = 400doppuu | site = ja.wikipedia.org}}
* {{target | user = スイス山梨県 | site = ja.wikipedia.org}}
* {{target | user = Tam0031 | site = ja.wikipedia.org}}
* {{target | user = 0-ZARA | site = ja.

## Upload the test list pages

In [99]:
test_strata["page_title"] = test_strata.apply(test_page_title, axis = 1)
test_strata.head()

,proj_group,edit_bin,sampled_users,survey_url,preview_url,page_title
0,arwiki,"[10, 30)","[[Neil P. Quinn-WMF, ar.wikipedia.org], [EGalv...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ar1
1,asia_wps,"[30, 150)","[[Neil P. Quinn-WMF, ko.wikipedia.org], [EGalv...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/as2
2,cee_wps,"[150, 600)","[[Neil P. Quinn-WMF, uk.wikipedia.org], [EGalv...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/ce3
3,commons,"[600, 1200)","[[Neil P. Quinn-WMF, commons.wikimedia.org], [...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/co4
4,dewiki,"[1200, 3500)","[[Neil P. Quinn-WMF, de.wikipedia.org], [EGalv...",https://wikimedia.qualtrics.com/jfe/form/SV_5A...,https://wikimedia.qualtrics.com/jfe/preview/SV...,User:Neil P. Quinn/2018 CE Insights/de5


In [100]:
for row in test_strata.itertuples(index = False):
    users = row.sampled_users
    
    # Only upload a list if the group has some sampled users
    if len(users) > 0:
        title = row.page_title
        text = list_text(users)
        comment = "Upload a test invitation list for the 2018 [[Community Engagement Insights]] survey"
        
        save_page(title, text, comment)

Page [[User:Neil P. Quinn/2018 CE Insights/ar1]] saved
Sleeping for 9.7 seconds, 2018-03-27 17:25:19
Page [[User:Neil P. Quinn/2018 CE Insights/as2]] saved
Sleeping for 9.7 seconds, 2018-03-27 17:25:30
Page [[User:Neil P. Quinn/2018 CE Insights/ce3]] saved
Sleeping for 9.5 seconds, 2018-03-27 17:25:40
Page [[User:Neil P. Quinn/2018 CE Insights/co4]] saved
Sleeping for 9.6 seconds, 2018-03-27 17:25:50
Page [[User:Neil P. Quinn/2018 CE Insights/de5]] saved
Sleeping for 9.2 seconds, 2018-03-27 17:26:00
Page [[User:Neil P. Quinn/2018 CE Insights/en6]] saved
Sleeping for 9.5 seconds, 2018-03-27 17:26:10
Page [[User:Neil P. Quinn/2018 CE Insights/es1]] saved
Sleeping for 9.5 seconds, 2018-03-27 17:26:20
Page [[User:Neil P. Quinn/2018 CE Insights/fr2]] saved
Sleeping for 9.6 seconds, 2018-03-27 17:26:30
Page [[User:Neil P. Quinn/2018 CE Insights/it3]] saved
Sleeping for 9.6 seconds, 2018-03-27 17:26:40
Page [[User:Neil P. Quinn/2018 CE Insights/ja4]] saved
Sleeping for 9.6 seconds, 2018-03-27

## Upload the list pages

<div class="alert alert-block alert-warning">
    <p>Running the cell below will create a lot of pages on Meta (or overwrite them if they've already been created).</p>
    
    <p>Run it only if you want to do that (but, if you run it accidentally, no major harm done).</p>
</div>

In [ ]:
for row in strata.itertuples(index = False):
    users = row.sampled_users
    
    # Only upload a list if the group has some sampled users
    if len(users) > 0:
        title = row.page_title
        text = list_text(users)
        comment = "Upload an invitation list for the 2018 [[Community Engagement Insights]] survey"
        
        # Uncomment this to really create all the pages
        # save_page(title, text, comment)

# Save our samples

In [60]:
# Have to serialize our arrays properly so we can reuse them in another file
strata["sampled_users"] = strata["sampled_users"].map(json.dumps)
strata.to_csv("sampled-strata.tsv", sep = "\t", index = False)

In [101]:
test_strata["sampled_users"] = test_strata["sampled_users"].map(json.dumps)
test_strata.to_csv("test-strata.tsv", sep = "\t", index = False)